# Evidencia 2. Reto: Modelado y simulación de una intersección víal empleando un sistema de multiagentes

Este modelo consta de una intersección de dos calles de doble sentido representada como una cuadricula discreta donde los carros deben esperar a la luz indicada del semaforo para poder avanzar. A su vez, lo carros tienen una probabilidad predeterminada (en este caso de 10%) de dar una vuelta a la derecha o izquierda dependiendo del sentido del carril en en el que se encuentren actualmente. Este sistema cuenta con 3 típos de agentes:

* Terreno: Ayudan a delimitar el espacio por el que pueden conducir los carros, evitando que estos se salgan del espacio de la calle.
* Semaforo: Cuentan con un conjunto de reglas que determinan su color en cada iteración del modelo. 
* Carro: Agente principal de la simulación, su movimiento está delimitado por la luz de los a su derecha (tomando el frente como la dirección en la que avanza).

Las reglas y funcionalidad de cada agente se encuentra en su subsección en el apartado de **Declaración de agentes**  

## Integrantes del equipo
* Luis Ángel Guzmán Iribe - A01741757
* Cesar Galvez - A01252177
* Antonio López Chávez - A01741741
* Sebastián Gálvez Trujillo - A01251884

## Profesores
* María Angélica Barreda Beltrán
* Jorge Mario Cruz Duarte

# Importación de librerías y clases para el modelo

In [1]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'MultiGrid' permite generar un "grid" que tenga la capacidad de tener más de un solo agente en una misma casilla
from mesa.space import SingleGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

from random import sample

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Imtesiones más esteticas y legibles para testeo
from pprint import pprint

# Para convertir diccionarios a JSON
import json 

# Declaración de agentes

## Terreno

Agente sin estado ni acciones que sirve para delimitar la calle. Representa el pasto y demás estructuras que rodea a la intersección

In [2]:
class terrainAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

## Semaforo

Agente sin acciones cuyo estado determina las acciones de los carros.

### Reglas de estado:
* Si no hay ningún carro con la misma orientación (vertical u horizontal) en la vecindad inmediata del semaforo, su color cambia a **amarillo**
* Si hay un carro en con la misma orientación en la vecindad inmediata del semáforo, y ninguno de los semáforos en una vecindad de un radio de 3 casillas con la misma orientación es verde, su color cambia a **Verde**
* Si hay un carro en con la misma orientación en la vecindad inmediata del semáforo, y al menos uno de los semáforos en una vecindad de un radio de 3 casillas con la misma orientación es verde, su color cambia a **Rojo**

Estas reglas se aplican constantemente con cada iteración del modelo

In [3]:
class trafficLightAgent(Agent):
    def __init__(self, unique_id, model, is_vertical):
        super().__init__(unique_id, model)
        self.colour = "yellow"
        self.vertical = is_vertical
        self.turn = False

    def rules(self):
        otherLights = self.model.grid.iter_neighbors(
            self.pos,
            moore=True,
            include_center=False,
            radius=3)

        nearbyCars = self.model.grid.iter_neighbors(
            self.pos,
            moore=True,
            include_center=False,
            radius=1)

        arrayLights = []
        for lights in otherLights:
            if isinstance(lights, trafficLightAgent):
                if lights.vertical != self.vertical:
                    arrayLights.append(lights)

        arrayCars = []
        for car in nearbyCars:
            if isinstance(car, carAgent):
                if self.vertical == car.vertical:
                    arrayCars.append(car)

        if len(arrayCars) > 0:
            for car in arrayCars:
                if self.checkGreenLights(arrayLights):
                    self.colour = "red"
                else:
                    self.colour = "green"
        else:
            self.colour = "yellow"

    def step(self):
        self.rules()

    def checkGreenLights(self, array):
        for lights in array:
            if lights.colour == "green":
                return True
        return False


## Carros

Principal agente del modelo, sus acciones y estado están determinadas por los agentes a su alrededor.

### Reglas de estado:
* Su orientación (dx, dy y vertical) se define en el momento que se crea la instancia del agente, pero puede cambiar mediante el método change_direction().
* Cuando se encuentra en medio de la intersección (sin vecinos de tipo terreno o semaforo) puede cambiar su orientación (a la del carril en el que se encuentra) con una probabilidad predeterminada, en este caso de 10%

### Reglas de acción:
* Si la casilla de adelante (dependiendo de la orientación del agente) está vacía, y no hay ninguún semaforo en su vecindad con la misma orientación: **avanza**.
* Si la casilla de adelante está vacía, y hay un semaforo en su vecindad con la misma orientación y estado en color verde: **avanza**.
* Si la casilla de adelante está vacía, y hay un semaforo en su vecindad con la misma orientación y estado en color rojo: **no avanza**.
* Si la casilla de adelante está obstruida por cualquier agente: **no avanza**.

In [4]:
class carAgent(Agent):
    def __init__(self, unique_id, model, probability_of_turning, dx, dy):
        super().__init__(unique_id, model)
        self.dx = dx
        self.dy = dy
        self.light = False
        self.probability_of_turning = probability_of_turning
        self.laps = 0

        if self.dy == 0:
            self.vertical = True
        else:
            self.vertical = False


    def move(self):
        posX, posY = self.pos
        next_pos = (posX + self.dx, posY + self.dy)

        # Regresar al inicio de la calle al acabar la ruta
        if self.model.grid.out_of_bounds(next_pos):
            self.next_pos = self.model.grid.torus_adj(next_pos)
            self.laps += 1
            self.light = False
        else:
            self.next_pos = next_pos

        # Posición en la que revisa si hay un semaforo que determine si puede avanzar
        checkPosX, checkPosY = self.pos
        if self.vertical:
            checkPosY -= self.dx
        else:
            checkPosX += self.dy
            
        checkPos = (checkPosX, checkPosY)

        # Obtiene contenido de coordenada de semaforo
        traffic_light_Cell = self.model.grid.get_cell_list_contents([checkPos])

        # Condicion para avanzar al ver semaforo
        if not self.model.grid.is_cell_empty(checkPos) and isinstance(traffic_light_Cell[0], trafficLightAgent) and not self.light:
            if traffic_light_Cell[0].colour == "green" and self.model.grid.is_cell_empty(self.next_pos):
                self.light = True
                self.model.grid.move_agent(self, self.next_pos)
        else:
            if self.model.grid.is_cell_empty(self.next_pos):
                self.model.grid.move_agent(self, self.next_pos)

    def turn(self):
        posX, posY = self.pos

        half_model_width = int(self.model.width/2)
        half_model_height = int(self.model.height/2)

        if self.vertical:
            # Carril de arriba
            if posX == half_model_width - 1 and posY == half_model_height - 1:
                self.change_direction(0, -1)
            elif posX == half_model_width - 1 and posY == half_model_height:
                self.change_direction(0, -1)

            # Carril de abajo
            elif posX == half_model_width and posY == half_model_height - 1:
                self.change_direction(0, 1)
            elif posX == half_model_width and posY == half_model_height:
                self.change_direction(0, 1)
        else:
            # Carril de arriba
            if posX == half_model_width - 1 and posY == half_model_height - 1:
                self.change_direction(1, 0)
            elif posX == half_model_width - 1 and posY == half_model_height:
                self.change_direction(-1, 0)

            # Carril de abajo
            elif posX == half_model_width and posY == half_model_height - 1:
                self.change_direction(1, 0)
            elif posX == half_model_width and posY == half_model_height:
                self.change_direction(-1, 0)

    def change_direction(self, dx, dy):
        self.dx = dx
        self.dy = dy

        if self.dy == 0:
            self.vertical = True
        else:
            self.vertical = False

    def step(self):
        
        neighbors = self.model.grid.iter_neighbors(
            self.pos,
            moore=False,
            include_center=False)

        neighbor_counter = 0

        for neighbor in neighbors:
            if not isinstance(neighbor, carAgent):
                neighbor_counter += 1

        if self.probability_of_turning >= np.random.random() and neighbor_counter == 0:
            self.turn() 

        self.move()

# Declaración del modelo

In [5]:
class streetModel(Model):
    def __init__(self, width=11, height=11, car_agents=4, probability_of_turning=0):
        self.grid = SingleGrid(width, height, True)
        self.width = width
        self.height = height
        self.schedule = SimultaneousActivation(self)

        """
        Genera los agentes 'semaforo' en las 4 esquinas de la intersección
        """

        t1 = trafficLightAgent(car_agents, self, True)
        self.grid.place_agent(
            t1,
            (int(width/2) - 2, int(height/2) - 2)
        )
        self.schedule.add(t1)

        t2 = trafficLightAgent(car_agents + 1, self, False)
        self.grid.place_agent(
            t2,
            (int(width/2) - 2, int(height/2) + 1)
        )
        self.schedule.add(t2)

        t3 = trafficLightAgent(car_agents + 2, self, False)
        self.grid.place_agent(
            t3,
            (int(width/2) + 1, int(height/2) - 2)
        )
        self.schedule.add(t3)

        t4 = trafficLightAgent(car_agents + 3, self, True)
        self.grid.place_agent(
            t4,
            (int(width/2) + 1, int(height/2) + 1)
        )
        self.schedule.add(t4)
        
        """
        Genera los agentes 'terreno' dejando 2 filas y 2 columnas en el centro de la cuadricula que actuarán como la calle
        """
        for (content, x, y) in self.grid.coord_iter():
            if not (x == int(width/2) or x == int(width/2) - 1 or y == int(height/2) or y == int(height/2) - 1) and content == None:
                a = terrainAgent((x, y), self)
                self.grid.place_agent(a, (x, y))

        """
        Genera los agentes 'carro' en las 2 filas y columnas que no fueron cubiertas por ajentes terrenos, inicializandolos en la orilla del tablero, 
        y colocando las nuevas instancias más cerca del centro si la posición más alejada del mismo ya está ocupada
        """
        offset = -1
        for i in range(car_agents):
            lane = i % 4

            if lane == 0:
                offset += 1
                c = carAgent(i, self, probability_of_turning, 0, 1)
                self.grid.place_agent(c, (int(height/2), offset))
            elif lane == 1:
                c = carAgent(i, self, probability_of_turning, 1, 0)
                self.grid.place_agent(c, (offset, int(width/2) - 1))
            elif lane == 2:
                c = carAgent(i, self, probability_of_turning, 0, -1)
                self.grid.place_agent(
                    c, (int(height/2) - 1, width - 1 - offset))
            elif lane == 3:
                c = carAgent(i, self, probability_of_turning, -1, 0)
                self.grid.place_agent(c, (height - 1 - offset, int(width/2)))
            self.schedule.add(c)


        """
        Función para recolección de datos
        """
        self.datacollector = DataCollector(
            model_reporters={
                "Grid": self.get_grid,
            },
            agent_reporters={
                "Cars": lambda a: {"dx": a.dx, "dy": a.dy, "id": a.unique_id, "laps": a.laps} if isinstance(a, carAgent) else 0,
                "TraficLights": lambda a: {"colour": a.colour, "id": a.unique_id} if isinstance(a, trafficLightAgent) else 0,
            }
        )


    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
    
    """
    Función auxiliar para guardar la cuadricula y las posiciones de los agentes.
    """

    def get_grid(self):
        grid = np.zeros((self.grid.width, self.grid.height))
        for tile in self.grid.coord_iter():
            tile_content, x, y = tile
            if isinstance(tile_content, terrainAgent):
                grid[x][y] = 0

            elif isinstance(tile_content, trafficLightAgent):
                if tile_content.colour == "yellow":
                    grid[x][y] = 1
                elif tile_content.colour == "green":
                    grid[x][y] = 2
                else:
                    grid[x][y] = 3

            elif isinstance(tile_content, carAgent):
                grid[x][y] = 4

            else:
                grid[x][y] = 5

        return grid


# Ejecución del modelo

In [10]:
# Definimos el ancho del Grid
GRID_WIDTH = 12

# Definimos la altura del Grid
GRID_HEIGHT = 12

# Cantidad de carros en la simulación
CAR_AGENTS = 6

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 300

PROBABILITY_OF_TURNING = 0.05

# Tiempo máximo para la ejecución (segundos y decimales de segundo [00.000000])
MAX_TIME = 0.010

start_time = time.time()
model = streetModel(GRID_WIDTH, GRID_HEIGHT, CAR_AGENTS, PROBABILITY_OF_TURNING)

for i in range(NUM_GENERATIONS):
        # print("STEP: ", i)
        model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

3 -> Ahora va a la izquierda
1 -> Ahora va a la izquierda
3 -> Ahora va hacia arriba
4 -> Ahora va hacia abajo
2 -> Ahora va hacia arriba
0 -> Ahora va hacia arriba
1 -> Ahora va hacia abajo
5 -> Ahora va a la izquierda
3 -> Ahora va a la izquierda
5 -> Ahora va hacia arriba
1 -> Ahora va a la izquierda
0 -> Ahora va a la izquierda
4 -> Ahora va a la derecha
1 -> Ahora va hacia arriba
3 -> Ahora va hacia arriba
3 -> Ahora va a la derecha
5 -> Ahora va a la izquierda
3 -> Ahora va hacia arriba
5 -> Ahora va hacia abajo
0 -> Ahora va hacia abajo
1 -> Ahora va a la izquierda
5 -> Ahora va a la derecha
1 -> Ahora va hacia arriba
2 -> Ahora va a la izquierda
1 -> Ahora va a la izquierda
0 -> Ahora va a la derecha
5 -> Ahora va hacia abajo
3 -> Ahora va a la derecha
1 -> Ahora va hacia arriba
4 -> Ahora va hacia arriba
Tiempo de ejecución: 0:00:00.122673


## Recolección de datos

In [11]:
carsDf = model.datacollector.get_agent_vars_dataframe().loc[:, ["Cars"]]

trafficLightsDf = model.datacollector.get_agent_vars_dataframe().loc[:, ["TraficLights"]]

steps = []

cars = [None] * CAR_AGENTS

prevIndex = 0

for index, row in carsDf.iterrows():
    # Esta row contiene un semaforo
    if (row.Cars == 0):
        continue

    if prevIndex == index[0]:
        cars[row.Cars["id"] - 1] = row.Cars
    else:
        prevIndex = index[0]
        steps.append({"cars": cars,  "traficLights": []})
        cars = [None] * CAR_AGENTS
        cars[row.Cars["id"] - 1] = row.Cars

steps.append({"cars": cars, "traficLights": []})

for index, row in trafficLightsDf.iterrows():
    if (row.TraficLights == 0):
        continue

    steps[index[0]]["traficLights"].append(row.TraficLights)

# pprint(steps)

with open('steps-data.json', 'w') as outfile:
    json.dump({"modelData": {"width": GRID_WIDTH, "height": GRID_HEIGHT,
              "carAgents": CAR_AGENTS, "numGenerations": NUM_GENERATIONS}, "steps": steps}, outfile)

all_grid = model.datacollector.get_model_vars_dataframe().loc[:, ["Grid"]]

# all_grid.to_json(r"./grid.json")


# Graficación del modelo

In [12]:
%%capture
fig, axs = plt.subplots(figsize=(8,8))
axs.set_xticks([])
axs.set_yticks([])
cmap = matplotlib.cm.get_cmap('viridis', 6) # Puede ser cualquier otra
cmap = cmap(np.linspace(0, 1, 6))
cmap[0] = np.array([121/256, 208/256, 33/256, 1]) # Verde - Terreno
cmap[1] = np.array([255/256, 255/256, 0/256, 1]) # Semaforo amarillo
cmap[2] = np.array([0/256, 256/256, 0/256, 1]) # Semaforo verde
cmap[3] = np.array([255/256, 0/256, 0/256, 1]) # Semaforo rojo
cmap[4] = np.array([0/256, 102/256, 204/256, 1]) # Azul - Carro
cmap[5] = np.array([160/256, 160/256, 160/256, 1]) # Gris - Calle

new_cmap = matplotlib.colors.ListedColormap(cmap)
patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [13]:
anim